# 要約 
このJupyterノートブックは、Kaggleの「LLM 20 Questions」コンペティションにおけるAIエージェントの作成プロセスを示しています。具体的には、AIが「20の質問」ゲームをプレイするためのエージェント（質問者および回答者）を構築しています。このノートブックでは、エラーが発生した際のメッセージ（Validation Episode failed）から始まっていますが、正しく実行されると、提出用の`submission.tar.gz`ファイルが生成されます。

### 主な課題
ノートブックは、AIエージェントに与えられたターゲット単語を推測させ、各エージェントが適切に連携しながら質問を行ったり、回答を提供したりする問題に取り組んでいます。質問者エージェントは質問を生成し、回答者エージェントは「はい」または「いいえ」で答えます。このダイナミックなやり取りを通じて、できるだけ早くターゲット単語を特定することが目指されます。

### 使用されている手法とライブラリ
ノートブックは以下の手法やライブラリを使用して問題にアプローチしています：

1. **Pythonライブラリのインストール**: 
   - `immutabledict`と`sentencepiece`を使用して、言語モデルのパフォーマンス向上を図っています。
   - GitHubから`gemma_pytorch`ライブラリをクローンし、AIエージェントに必要な設定を整えています。

2. **GemmaFrameworkの使用**: 
   - `GemmaForCausalLM`を利用して言語モデルを構築し、AIエージェントを効果的に機能させるための基盤を提供しています。

3. **エージェントクラスの定義**:
   - `GemmaAgent`クラスをベースに、質問者エージェント（`GemmaQuestionerAgent`）と回答者エージェント（`GemmaAnswererAgent`）を定義。
   - それぞれのクラスは、ユーザーからの入力を受け取り、ターゲットに基づいて動的に応答を生成します。

4. **プロンプトフォーマッティング**:
   - `GemmaFormatter`クラスを使用して、ユーザーとモデル間の会話の状態を管理し、適切にフォーマットされたプロンプトを生成します。

5. **圧縮とファイル管理**:
   - `pigz`と`pv`を利用して作成した圧縮ファイルを用いて、エージェントをKaggleに提出するための準備を整えています。これにより、ファイルの圧縮速度が向上し、その進行状況を確認することができます。

このノートブック全体を通じて、KaggleのコンペティションにおけるLLMの適用と、AIエージェントの構築手法が詳細に示されています。成功すれば、AIは「20の質問」ゲームにおいて高いパフォーマンスを発揮することが期待されます。

---


# 用語概説 
以下は、Jupyterノートブックに関連する機械学習・深層学習の専門用語に関する解説です。初心者がつまずきそうな専門用語に焦点を当てています。

1. **LLM（Large Language Model）**:
   - 大規模言語モデルの略称。トレーニングデータセットに大量のテキストデータを使用して訓練されたモデルで、自然言語処理において非常に高い性能を発揮します。通常、数億から数十億のパラメータを持ち、文脈を理解する能力があります。

2. **Gemma**:
   - Kaggleのコンペティションで使用される特定のモデル名です。このモデルは、効率的な言語生成を目的としたアーキテクチャに基づいており、特に因果的言語モデル（Causal Language Model）として設計されています。

3. **few-shot examples**:
   - モデルに与えて学習を助けるための少数の例。特に少数ショット学習は、十分なトレーニングデータがない場合でもモデルがタスクを学習するのに役立つ手法です。この手法は、新しいコンテキストでの応答を生成する際に非常に有用です。

4. **プロンプト**:
   - モデルに対して与えられる指示や質問のこと。言語モデルがどのように応答するかを決定するため、プロンプトの設計が重要です。システムプロンプトは特に、モデルがなすべき役割を定義するために使用されます。

5. **contextlib**:
   - Pythonの標準ライブラリの一部で、コンテキストマネージャを簡単に作成するための機能を提供します。リソース管理や一時的な設定のためにしばしば使用され、リソースの解放や状態の復元を自動で行います。

6. **torch.device**:
   - PyTorchにおけるデバイスの指定に使用されるクラス。計算をどのハードウェア（CPUまたはGPU）で行うかを示します。例えば、`cuda:0`は最初のGPUを指します。

7. **量子化（Quantization）**:
   - モデルのパラメータを減少させ、モデルのサイズを最適化し、推論速度を向上させるために数値の精度を下げる技術。これにより、リソースの限られたデバイスでも大規模なモデルを使用できるようになります。

8. **itertools**:
   - Pythonのライブラリで、効率的なループや組み合わせを生成するためのツールが含まれています。特に、`zip_longest`関数は、異なる長さのリストを組み合わせる際に使用され、リストの要素が不足している場合は`None`で埋めます。

9. **状態管理（State Management）**:
   - モデルやシステムの現在の状態を追跡・保存する技術。例えば、会話の流れやユーザーからの入力を保持することで、コンテキストを維持します。

10. **アサーション（Assertion）**:
    - プログラムの実行時に特定の条件が満たされているかを確認するための手法。条件が満たされない場合、エラーメッセージを表示し、プログラムを停止します。デバッグやテストに有用です。

これらの用語は、ノートブックの実装において重要な役割を果たしており、理解しておくことが各コンポーネントの機能や相互作用を理解する助けになります。

---


2024年5月20日

* **ノートブックを提出しようとしましたが、*Validation Episode failed*というエラーが返ってきました。**

このノートブックは、**LLM 20 Questions**のエージェント作成プロセスを示しています。このノートブックを実行すると、`submission.tar.gz`ファイルが生成されます。ノートブックビューアから、*Output*タブをクリックし、`submission.tar.gz`を見つけてダウンロードしてください。コンペティションのホームページの左上にある**Submit Agent**をクリックして、ファイルをアップロードして提出してください。 




In [ ]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

**上記のコードの説明**

このコードは、作業環境を設定し、必要なPythonライブラリをインストールし、特定のライブラリ（GitHubリポジトリから）を使用するための準備を行うBashスクリプトです。以下は各行の詳細な説明です：
1. `%%bash`:
   - これはJupyterノートブックで使われるマジックコマンドで、以下の行がBashコマンドとして解釈されることを示します。
2. `cd /kaggle/working`:
   - 現在のディレクトリを`/kaggle/working`に変更します。このディレクトリは、Kaggleカーネル（ノートブック）で計算中にファイルを保存およびアクセスするためによく使用されます。
3. `pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece`:
   - `pip`を使用して、`immutabledict`と`sentencepiece`の最新バージョンを静かに（`-q`）インストールします（`-U`はアップグレードの意味）。
   - `-t /kaggle/working/submission/lib`オプションは、ライブラリをインストールする対象ディレクトリを指定します。
4. `git clone https://github.com/google/gemma_pytorch.git > /dev/null`:
   - GitHubから`gemma_pytorch`リポジトリをクローンします。
   - `> /dev/null`部分は、出力を`/dev/null`にリダイレクトし、クローンコマンドの出力をサイレントにしてノートブックやスクリプトの出力をきれいに保ちます。
5. `mkdir /kaggle/working/submission/lib/gemma/`:
   - `/kaggle/working/submission/lib/`内に、`gemma`という名前の新しいディレクトリを作成します。このディレクトリは、クローンしたリポジトリのファイルを保存するために使用されます。
6. `mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/`:
   - クローンした`gemma_pytorch`リポジトリ内の`gemma`ディレクトリからすべてのファイル（`*`）を新しく作成した`gemma`ディレクトリに移動します。
### まとめ
- このスクリプトは作業ディレクトリを設定します。
- `immutabledict`と`sentencepiece`ライブラリを特定のパスにインストールします。
- GitHubからリポジトリをクローンします。
- クローンしたリポジトリのファイルを格納するためのディレクトリを作成します。
- 必要なファイルを新しく作成したディレクトリに移動します。
この設定は、機械学習タスクやKaggleノートブックでのその他の計算タスクに必要な特定の依存関係とライブラリを準備するために使用される可能性があります。


In [ ]:
%%writefile submission/main.py
# セットアップ
import os
import sys

# **重要:** コードがノートブックとシミュレーション環境の両方で動作するように、システムパスをこのように設定してください
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトフォーマット
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# エージェント定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """指定されたdtypeにデフォルトのtorch dtypeを設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルの初期化中")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        # モデルを呼び出して応答を生成します
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        # 応答から**で囲まれたキーワードを見つけ出します
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問を始めましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("キーワードを予想してください。予想を**で囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "それは人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("未知のターンタイプ:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問を始めましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}です。カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"キーワード{obs.keyword}に関する質問です。はいまたはいいえで答えてください。答えを**で囲んでください**。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        # 応答から解析したキーワードに基づいて「はい」または「いいえ」を返します
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

**上記のコードの説明**

このスクリプトは、20の質問ゲームをプレイするために設計されたAIエージェントのセットアップと動作を定義します。エージェントは質問者または回答者のいずれかになります。コードの各部分についての詳細な説明は以下の通りです：
### 1. ファイル作成
```python
%%writefile submission/main.py
```
- このJupyterノートブックのマジックコマンドは、以下のコードを`submission`ディレクトリ内の`main.py`というファイルに書き込みます。
### 2. セットアップ
```python
import os
import sys
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")
```
- 必要なモジュールをインポートします。
- システムパスを指定して、ノートブックとKaggleシミュレーション環境の両方で必要なライブラリが利用できるようにします。
### 3. ライブラリおよび設定のインポート
```python
import contextlib
from pathlib import Path
import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM
if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"
```
- 追加のライブラリをインポートします。
- 環境に基づいてモデルの重みのパスを定義します。
### 4. プロンプトフォーマッティングクラス
```python
import itertools
from typing import Iterable
class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()
    def __repr__(self):
        return self._state
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self
    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self
    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self
```
- `GemmaFormatter`は、ユーザーとモデル間の会話をフォーマットする役割を果たします。会話のターンの開始と終了を示すために定義されたトークンを使用します。
- ユーザーやモデルのプロンプトを追加したり、ターンの開始と終了を管理したり、会話の状態をリセットするためのメソッドがあります。
### 5. エージェント定義
```python
import re
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)
class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)
        print("モデルの初期化中")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()
    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response
    def _start_session(self, obs: dict):
        raise NotImplementedError
    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
                }
        # モデルを呼び出して応答を生成します
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response
    def _parse_keyword(self, response: str):
        # 応答から**で囲まれたキーワードを見つけ出します
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword
    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError
def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]
```
- `GemmaAgent`クラスは、モデルを初期化し、プロンプトをフォーマットし、モデルとのインタラクションを処理します。
- `_set_default_tensor_type`コンテキストマネージャは、PyTorchのデフォルトテンソル型を一時的に設定します。
- `__call__`メソッドは、観察値（`obs`）に基づいてモデルから応答を生成します。
- `_start_session`と`_parse_response`メソッドは、サブクラスによって実装されることを想定しているプレースホルダです。
- `interleave_unequal`関数は、2つのリストから要素を交互に並べ、それぞれのリストに多くの要素が含まれている場合は残りの要素で埋めます。
### 6. 質問者と回答者エージェント
```python
class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("20の質問を始めましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("キーワードを予想してください。予想を**で囲んでください。")
        self.formatter.start_model_turn()
    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "それは人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("未知のターンタイプ:", obs.turnType)
class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"20の質問を始めましょう。あなたは回答者の役割を果たします。キーワードは{obs.keyword}です。カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"キーワード{obs.keyword}に関する質問です。はいまたはいいえで答えてください。答えを**で囲んでください**。")
        self.formatter.start_model_turn()
    def _parse_response(self, response: str, obs: dict):
        # 応答から解析したキーワードに基づいて「はい」または「いいえ」を返します
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'


In [ ]:
# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者からのはい/いいえ質問に応じます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問を始めましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** それではキーワードを推測してください。",
    "**フランス**", "正解です！",
]


# **重要:** エージェントはグローバルに定義して、必要なエージェントのみをロードします。
# 両方のエージェントを読み込むと、OOM（メモリ不足）になる可能性があります。
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    # エージェントの役割に基づいて処理を呼び出します
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "はい"
    else:
        return response

**上記のコードの説明**
このコード部分は、20の質問ゲームをプレイするために使用されるAIエージェントの作成と管理を担当しています。エージェントをインスタンス化する方法と、提供された観察に基づいてゲームロジックを処理する方法を定義します。以下は詳細な説明です：
### 1. システムプロンプトと少数ショットの例の定義
```python
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、回答者がキーワードを考え、質問者からのはい/いいえ質問に応じます。キーワードは特定の人、場所、または物です。"
few_shot_examples = [
    "20の質問を始めましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "それは人ですか？", "**いいえ**",
    "それは場所ですか？", "**はい**",
    "それは国ですか？", "**はい** それではキーワードを推測してください。",
    "**フランス**", "正解です！",
]
```
- `system_prompt`: AIに20の質問ゲームについてのコンテキストを提供します。
- `few_shot_examples`: AIがゲームの形式と応答の仕方を理解するのに役立つ一連の例を示します。この少数ショット学習は、モデルがいくつかの例から一般化するのに役立ちます。
### 2. グローバルエージェント変数
```python
agent = None
```
- 現在のエージェントを保存するためのグローバル変数`agent`が定義されています。これにより、エージェントを複数回ロードせずに済み、メモリ不足（OOM）エラーを回避します。
### 3. 適切なエージェントを取得するための関数
```python
def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"
    return agent
```
- `get_agent(name: str)`: 指定されたエージェント（`questioner`または`answerer`）のインスタンスを返す関数です。
    - グローバルな`agent`変数が`None`であり、リクエストされたエージェントタイプが`questioner`の場合、`GemmaQuestionerAgent`のインスタンスを作成します。
    - 同様に、`answerer`の場合は`GemmaAnswererAgent`のインスタンスを作成します。
    - 両方のエージェントタイプは、`system_prompt`と`few_shot_examples`を使って初期化されます。
    - エージェントが必ず返されることを保証し、初期化を試みた後に`agent`が`None`でないことを確認します。
### 4. 主なエージェント関数
```python
def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)
    if response is None or len(response) <= 1:
        return "はい"
    else:
        return response
```
- `agent_fn(obs, cfg)`: ゲームの現在の状態（`obs`）に基づいてエージェントとのインタラクションを処理する主関数です。
    - `obs.turnType`が`"ask"`の場合、`questioner`エージェントを呼び出して質問を生成します。
    - `obs.turnType`が`"guess"`の場合、`questioner`エージェントを呼び出して予想を生成します。
    - `obs.turnType`が`"answer"`の場合、`answerer`エージェントを呼び出して答えを生成します。
    - 必要に応じて`get_agent`を使用してエージェントを取得します。
    - エージェントの応答が`None`または非常に短い場合、"はい"を返すようにデフォルト化します。
    - そうでなければ、エージェントが生成した応答を返します。
### まとめ
- このコードセグメントは、20の質問ゲームをプレイするためのAIエージェントを初期化し管理します。
- エージェントのためのシステムプロンプトと例を定義します。
- メモリの問題を防ぐために、エージェントが1つだけロードされるようにします。
- `get_agent`関数は必要なエージェントタイプを動的に初期化します。
- `agent_fn`関数は、現在のターンタイプに基づいてエージェントを呼び出すゲームロジックを処理します。


In [ ]:
!apt install pigz pv > /dev/null

**上記のコードの説明**
この1行のコードは、`apt`パッケージマネージャを用いて`pigz`と`pv`の2つのパッケージをインストールするシェルコマンドです。以下は詳細な説明です：
### コマンドの内訳
```bash
!apt install pigz pv > /dev/null
```
#### 1. `!`
- 行の先頭にある`!`は、このコマンドがシェルで実行されるべきであることを示します。これは Jupyter ノートブックでの特徴で、コードセルから直接シェルコマンドを実行できます。
#### 2. `apt install pigz pv`
- `apt`: これは、Debian系Linuxディストリビューション（例：Ubuntu）で使用されるパッケージ管理システムであるAdvanced Package Tool（APT）のコマンドラインインターフェースです。
- `install`: このサブコマンドは、1つまたは複数のパッケージをインストールしたいことをaptに指示します。
- `pigz`: インストールする最初のパッケージの名前です。`pigz`は「gzipの並列実装」を意味します。これは、データを圧縮するためのツールであり、従来の`gzip`よりも速くデータを圧縮できます。
- `pv`: インストールする2番目のパッケージの名前です。`pv`は「Pipe Viewer」を意味します。これは、パイプを介るデータの進行状況を監視できる端末ベースのツールです。データ転送の進捗、スループット率、および完了までの推定時間を表示します。
#### 3. `> /dev/null`
- このコマンドの部分は、標準出力（stdout）を`/dev/null`にリダイレクトします。
- `/dev/null`は、Unix系オペレーティングシステムにおける特別なファイルであり、書き込まれたすべてのデータを破棄します。通常、表示したくない出力を抑制するために使用されます。
- この文脈では、`> /dev/null`により、`apt install`コマンドによって生成されたいかなる出力（例：ダウンロードの進行状況、インストールメッセージ）が端末やJupyterノートブックに表示されないようにします。
### コマンドの目的
このコマンドの目的は、`pigz`と`pv`パッケージを静かにインストールすることです。出力を表示せずにインストールを実行することは、スクリプトやノートブックセルにおいて、インストールログで出力が混雑することを防ぐのに便利です。
### 実用例
- **`pigz`**: 大きなファイルを迅速に圧縮する際に、複数のCPUコアを活用できるので便利です。
- **`pv`**: 長時間のプロセスにおいてデータの進行状況を監視する際に役立ちます。
このコマンドを利用することで、後の処理のためにこれらのツールをシステムに利用可能にしつつ、インストールメッセージで出力を混乱させないようにしています。


In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2

**上記のコードの説明**
このシェルコマンドは、`pigz`と`pv`の助けを借りて、圧縮されたtarボール（`submission.tar.gz`）を作成します。以下はコマンドの各部分の詳細な内訳です：
### コマンドの内訳
```bash
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2
```
#### 1. `!`
- この文字は、次のコマンドがシェルで実行されるべきであることを示しています。これはJupyterノートブックで利用可能な機能です。
#### 2. `tar`
- `tar`コマンドは、アーカイブの作成、維持、変更、および抽出を行うために使用されます。
#### 3. `--use-compress-program='pigz --fast --recursive | pv'`
- このオプションは使用する圧縮プログラムを指定します。この場合は、`pigz`を`pv`でパイプで接続します。
- `'pigz --fast --recursive'`: `pigz`（gzipの並列実装）を使用して圧縮します。`--fast`フラグは、圧縮率よりも速度を優先するように指示し、`--recursive`は、ディレクトリを再帰的に処理することを保証します。
- `'| pv'`: `pigz`の出力を`pv`にパイプします。これにより圧縮の進行を監視できます。
#### 4. `-cf submission.tar.gz`
- `-c`: 新しいアーカイブを作成します。
- `-f submission.tar.gz`: 作成するアーカイブファイルの名前を指定します（`submission.tar.gz`）。
#### 5. `-C /kaggle/working/submission .`
- `-C /kaggle/working/submission`: アーカイブにファイルを追加する前にディレクトリを`/kaggle/working/submission`に変更します。
- `.`: 現在のディレクトリ（今は`/kaggle/working/submission`）をアーカイブに追加します。
#### 6. `-C /kaggle/input/ gemma/pytorch/7b-it-quant/2`
- `-C /kaggle/input/`: アーカイブに追加する前に、`/kaggle/input/`ディレクトリに変更します。
- `gemma/pytorch/7b-it-quant/2`: `/kaggle/input/`内の`gemma/pytorch/7b-it-quant/2`ディレクトリをアーカイブに追加します。
### コマンドの目的
このコマンドは、以下を含む圧縮tarボール（`submission.tar.gz`）を作成するために使用されます：
1. `/kaggle/working/submission`ディレクトリ内のすべてのファイルとディレクトリ。
2. `/kaggle/input/`からの`gemma/pytorch/7b-it-quant/2`ディレクトリ。
`pigz`を使用して圧縮することで、マルチCPUコアによるより高速な圧縮を実現し、`pv`が圧縮の進行を可視化します。
### 実用例
- **圧縮速度**: `pigz`は複数のCPUコアを活用することで、圧縮プロセスを加速させます。
- **進行状況の監視**: `pv`は圧縮進行状況を監視し、プロセスの進捗を視覚的に示すのに役立ちます。
- **特定のファイルのアーカイブ**: `-C`オプションを使用することで、現在の作業ディレクトリを変更することなく、アーカイブに特定のディレクトリを含めることができ、柔軟に複数の場所からファイルを追加されます。


---

# コメント 

> ## Matin Mahmoudi ✨
> 
> 素晴らしいスタートノートブックで、実装に関する情報が豊富です [@regisvargas](https://www.kaggle.com/regisvargas)。あなたの他の作品もチェックし、すべてに投票しました。すごい、友達！
> 
> 
> 
> > ## Regis Vargas（トピック作成者）
> > 
> > とても励みになるお言葉をありがとうございます。私は多くの困難に直面していますが、日々自分を押し上げる努力をしています。
> > 
> > 
> > 

---

